In [ ]:
import volstreet.datamodule as dm
import plotly.express as px
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from datetime import time, datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import volstreet as vs

In [ ]:
# Using DataClient class
client = dm.DataClient(api_key=__import__('os').environ['EOD_API_KEY'])
kite_user = __import__('os').environ['KITE_USER']
kite_pass = __import__('os').environ['KITE_PASS']
kite_api_key = __import__('os').environ['KITE_API_KEY']
kite_api_secret = __import__('os').environ['KITE_API_SECRET']
kite_auth_key = __import__('os').environ['KITE_AUTH_KEY']

In [ ]:
# Using get_data and analyser functions
nifty_data = client.get_data(symbol='NIFTY')
bnf_data = client.get_data(symbol='BANKNIFTY')
finnifty_data = client.get_data(symbol='FINNIFTY')
nifty_daily_data = dm.analyser(nifty_data, frequency='D')
bnf_daily_data = dm.analyser(bnf_data, frequency='D')
nifty_weekly_data = dm.analyser(nifty_data, frequency='W-THU')
bnf_weekly_data = dm.analyser(bnf_data, frequency='W-THU')
nifty_monthly_data = dm.analyser(nifty_data, frequency='M-THU')
bnf_monthly_data = dm.analyser(bnf_data, frequency='M-THU')

In [ ]:
# Using ratio_analysis function
rolling_periods = 5
ratio_data = dm.ratio_analysis(bnf_weekly_data, nifty_weekly_data, add_rolling=rolling_periods)
px.line(ratio_data, x=ratio_data.index, y=['BANKNIFTY Change', 'NIFTY Change', f'Rolling {rolling_periods} Ratio'], color_discrete_map={'BANKNIFTY Change': 'red', 'NIFTY Change': 'blue', f'Rolling {rolling_periods} Ratio': 'green'})

In [ ]:
ratio_data

In [ ]:
# Using gambler function for NIFTY and BANKNIFTY
for index in ['NIFTY', 'BANKNIFTY']:
    print(f'{index}\n')
    data = client.get_data(symbol=index)
    for frequency in ['D', 'D-THU', 'W-THU', 'M-THU']:
        print(f'{frequency}\n')
        dm.gambler(data, frequency, 'abs_change')

In [ ]:
# Using gambler function for FINNIFTY
for index in ['FINNIFTY']:
    print(f'{index}\n')
    data = client.get_data(symbol=index)
    for frequency in ['D', 'D-TUE', 'W-TUE', 'M-TUE']:
        print(f'{frequency}\n')
        dm.gambler(data, frequency, 'abs_change')

In [ ]:
# Rolling average of absolute change to support gambler function
analysed_df = dm.analyser(finnifty_data, frequency='W-tue')
rolling_periods = 9
analysed_df['rolling'] = analysed_df['abs_change'].rolling(rolling_periods, min_periods=1).mean()
px.line(analysed_df, x=analysed_df.index, y='rolling')

In [ ]:
analysed_df

# One min data

In [ ]:
kite_obj = dm.get_greenlit_kite(kite_api_key, kite_api_secret, kite_user, kite_pass, kite_auth_key)

In [ ]:
dm.get_1m_data(kite_obj, 'SBILIFE', path='data\\')

In [ ]:
# Updating one min data for NIFTY 50, NIFTY BANK and NIFTY FIN SERVICE
dm.get_1m_data(kite_obj, 'NIFTY 50', path='data\\')
dm.get_1m_data(kite_obj, 'NIFTY BANK', path='data\\')
dm.get_1m_data(kite_obj, 'NIFTY FIN SERVICE', path='data\\')
dm.get_1m_data(kite_obj, 'NIFTY MID SELECT', path='data\\')

In [ ]:
dm.get_constituent_1m_data(kite_obj, 'NIFTY', path='data\\')

# Intraday Trend

In [ ]:
nifty_onemin = pd.read_csv('data/NIFTY 50_onemin_prices.csv', index_col=0, parse_dates=True)
bnf_onemin = pd.read_csv('data/NIFTY BANK_onemin_prices.csv', index_col=0, parse_dates=True)
fin_onemin = pd.read_csv('data/NIFTY FIN SERVICE_onemin_prices.csv', index_col=0, parse_dates=True)
midcp_onemin = pd.read_csv('data/NIFTY MID SELECT_onemin_prices.csv', index_col=0, parse_dates=True)

In [ ]:
trend_nifty = dm.backtest_intraday_trend(nifty_onemin, open_nth = 0, beta = 0.8, eod_client=client, max_entries=3)
trend_bnf = dm.backtest_intraday_trend(bnf_onemin, open_nth = 0, beta = 0.8, eod_client=client, max_entries=3)
trend_finnifty = dm.backtest_intraday_trend(fin_onemin, open_nth = 0, beta = 0.8, eod_client=client, max_entries=3)

In [ ]:
trend_bnf

In [ ]:
# Plotting the distribution of returns for various entries
df_to_plot = trend_finnifty
returns_1 = df_to_plot.trade_data.apply(lambda x: x.get('entry_1', {}).get('returns', np.nan)).dropna()
returns_2 = df_to_plot.trade_data.apply(lambda x: x.get('entry_2', {}).get('returns', np.nan)).dropna()
returns_3 = df_to_plot.trade_data.apply(lambda x: x.get('entry_3', {}).get('returns', np.nan)).dropna()

fig = go.Figure()
fig.add_trace(go.Histogram(x=returns_1, name='Entry 1', nbinsx=10))
fig.add_trace(go.Histogram(x=returns_2, name='Entry 2', nbinsx=10))
fig.add_trace(go.Histogram(x=returns_3, name='Entry 3', nbinsx=10))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
# Year wise summary of returns
df_to_sum = trend_nifty
df_to_sum.groupby(df_to_sum.index.year).total_returns.sum()

In [ ]:
all_indices_returns = trend_bnf.merge(trend_finnifty, left_index=True, right_index=True, suffixes=('_bnf', '_finnifty')).merge(trend_nifty, left_index=True, right_index=True, suffixes=('', '_nifty'))[['total_returns', 'total_returns_bnf', 'total_returns_finnifty']]
all_indices_returns

In [ ]:
# Plotting the distribution of the ratio for stop loss and no stop loss days for all indices
all_indices_with_drivers = pd.concat([trend_nifty, trend_bnf, trend_finnifty])
all_indices_with_drivers_stop_loss =  all_indices_with_drivers[(all_indices_with_drivers.total_returns <= 0)]
all_indices_with_drivers_no_stop_loss =  all_indices_with_drivers[(all_indices_with_drivers.total_returns > 0)]
fig = px.histogram(all_indices_with_drivers_stop_loss, x='ratio')
fig.add_trace(go.Histogram(x=all_indices_with_drivers_no_stop_loss.ratio, name='No Stop Loss'))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)

In [ ]:
# Plotting the minute vol and open to close trend on different y axis
df_to_plot = trend_finnifty
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Line(x=df_to_plot.index, y=df_to_plot['rolling_ratio'], name='Ratio'), secondary_y=True)
fig.add_trace(go.Line(x=df_to_plot.index, y=df_to_plot['strat_nav'], name='Nav'), secondary_y=False)

In [ ]:
# Trying different beta values
for beta in range(80, 105, 5):
    beta = beta/100
    trend_nifty = dm.backtest_intraday_trend(nifty_onemin, beta = beta, eod_client=client)
    print(f'Beta: {beta}')
    print(f'NIFTY: {trend_nifty["total_returns"].sum()}')

# Intraday Trend - Constituent analysis

In [ ]:
def get_index_with_constituent_trend_data(index_name, trend_df):

    index_onemin = pd.read_csv(f'data/{index_name}_onemin_prices.csv', index_col=0, parse_dates=True)
    index_daily_open = (index_onemin.groupby(index_onemin.index.date).apply(lambda x: x.iloc[1]).open.to_frame())
    index_onemin['day_open'] = index_daily_open.loc[index_onemin.index.date].values
    index_onemin['change_from_open'] = index_onemin['close'] / index_onemin['day_open'] - 1
    index_onemin = index_onemin[['change_from_open']]
    index_onemin.columns = map(lambda x: f'{index_name}_{x}', index_onemin.columns)
    tickers, weights = vs.get_index_constituents(index_name)
    ticker_dfs = []
    for ticker, weight in zip(tickers, weights):
        ticker_onemin = pd.read_csv(f'data/{ticker}_onemin_prices.csv', index_col=0, parse_dates=True)
        ticker_onemin['weight'] = weight/100
        ticker_daily_open = (ticker_onemin.groupby(ticker_onemin.index.date).apply(lambda x: x.iloc[1]).open.to_frame())
        ticker_onemin['day_open'] = ticker_daily_open.loc[ticker_onemin.index.date].values
        ticker_onemin['change_from_open'] = ticker_onemin['close'] / ticker_onemin['day_open'] - 1
        ticker_onemin['weighted_change'] = ticker_onemin['change_from_open'] * ticker_onemin['weight']
        ticker_onemin = ticker_onemin[['change_from_open', 'weighted_change']]
        ticker_onemin.columns = map(lambda x: f'{ticker}_{x}', ticker_onemin.columns)
        ticker_dfs.append(ticker_onemin)
    full_df = pd.concat(ticker_dfs, axis=1)
    full_df['proxy_index_change'] = full_df.filter(regex='weighted_change').sum(axis=1)
    full_df = full_df.merge(index_onemin, left_index=True, right_index=True)
    for ticker in tickers:
        full_df[f'{ticker}_contribution'] = full_df[f'{ticker}_weighted_change'] / full_df[f'proxy_index_change']
        full_df[f'{ticker}_contribution_sq'] = full_df[f'{ticker}_contribution'] ** 2
    _trigger_times = [day[entry]['trigger_time'] for day in trend_df.trade_data for entry in day.keys() if entry != 'total_returns']
    _returns = [day[entry]['returns'] for day in trend_df.trade_data for entry in day.keys() if entry != 'total_returns']
    _trend_at_close = [trend_df.set_index(trend_df.index.date).loc[tt.date()].open_to_close_trend for tt in _trigger_times]
    _trigger_returns_trend = pd.DataFrame({'trigger_time': _trigger_times, 'returns': _returns, 'trend_at_close': _trend_at_close})
    df_to_ret = full_df.merge(_trigger_returns_trend, left_index=True, right_on='trigger_time')
    df_to_ret['sum_of_abs_movement'] = df_to_ret.drop(columns=[f'{index_name}_change_from_open']).filter(regex='change_from_open').abs().sum(axis=1)
    df_to_ret['std_of_ratio'] = df_to_ret.drop(columns=[f'{index_name}_change_from_open']).filter(regex='change_from_open').div(df_to_ret['sum_of_abs_movement'], axis=0).std(axis=1)
    df_to_ret['std_of_constituents'] = df_to_ret.drop(columns=[f'{index_name}_change_from_open']).filter(regex='change_from_open').std(axis=1)
    df_to_ret['hhi_index'] = df_to_ret.filter(regex='contribution_sq').sum(axis=1)


    return df_to_ret

In [ ]:
trend_bnf_consolidated = get_index_with_constituent_trend_data('NIFTY BANK', trend_bnf)
trend_bnf_consolidated_post_2021 = trend_bnf_consolidated[trend_bnf_consolidated.trigger_time > datetime(2021, 1, 1)]
trend_bnf_consolidated_post_2021

In [ ]:
trend_nifty_consolidated = get_index_with_constituent_trend_data('NIFTY 50', trend_nifty)

In [ ]:
trend_nifty_consolidated_post_2023 = trend_nifty_consolidated[trend_nifty_consolidated.trigger_time > datetime(2023, 1, 1)]
trend_nifty_consolidated_post_2023

In [ ]:
px.scatter(trend_bnf_consolidated_post_2021, x='std_of_constituents', y='std_of_ratio', hover_data=['trigger_time', 'returns', 'trend_at_close'], color='returns', range_color=[-0.3, 1])

# Index flat vs constituents move

In [ ]:
bnf_index_vs_cons =  dm.get_index_vs_constituents_recent_vols('BANKNIFTY', return_all=False, simulate_backtest=True)

In [ ]:
bnf_index_vs_cons

# Insights

In [ ]:
# Confirming that there is a certain drift in absolute changes as time frame increases

for index_name, daily_df in zip(['NIFTY', 'BANKNIFTY', 'FINNIFTY'], [nifty_data, bnf_data, finnifty_data]):

    daily_vol = daily_df.resample('B').ffill().close.pct_change().abs().mean()
    weekly_vol = daily_df.resample('W').ffill().close.pct_change().abs().mean()
    monthly_vol = daily_df.resample('M').ffill().close.pct_change().abs().mean()
    yearly_vol = daily_df.resample('Y').ffill().close.pct_change().abs().mean()

    weekly_ratio = weekly_vol / daily_vol
    monthly_ratio = monthly_vol / daily_vol
    yearly_ratio = yearly_vol / daily_vol

    weekly_benchmark = 5**0.5
    monthly_benchmark = 21**0.5
    yearly_benchmark = 252**0.5

    weekly_deviation_from_benchmark = weekly_ratio/weekly_benchmark
    monthly_deviation_from_benchmark = monthly_ratio/monthly_benchmark
    yearly_deviation_from_benchmark = yearly_ratio/yearly_benchmark

    print(f'{index_name}\nDaily Volatility: {daily_vol:0.3f}\nWeekly Volatility: {weekly_vol: 0.3f}, Weekly Ratio: {weekly_ratio: 0.3f}, Weekly Benchmark: {weekly_benchmark: 0.3f}\nMonthly Volatility: {monthly_vol: 0.3f}, Monthly Ratio: {monthly_ratio: 0.3f}, Monthly Benchmark: {monthly_benchmark: 0.3f}\nYearly Volatility: {yearly_vol: 0.3f}, Yearly Ratio: {yearly_ratio: 0.3f}, Yearly Benchmark: {yearly_benchmark: 0.3f}\n')

    print(f'{index_name}\nWeekly Deviation from Benchmark: {weekly_deviation_from_benchmark: 0.3f}\nMonthly Deviation from Benchmark: {monthly_deviation_from_benchmark: 0.3f}\nYearly Deviation from Benchmark: {yearly_deviation_from_benchmark: 0.3f}\n')

In [ ]:
# Confirming whether drift is present in intraday movements

for onemindf, index_name in zip([nifty_onemin, bnf_onemin, fin_onemin, midcp_onemin], ['NIFTY', 'BANKNIFTY', 'FINNIFTY', 'MIDCAP']):
    print(f'{index_name}\n')

    filtered_index = filter(lambda i: i.time() not in [time(9, 15), time(9, 16), time(15, 30)], onemindf.index)
    filtered_index = list(filtered_index)

    minute_vol_sd = onemindf.close.pct_change()[filtered_index].std()
    minute_vol_abs_change = onemindf.close.pct_change()[filtered_index].abs().mean()


    print(f'Minute Volatility SD: {minute_vol_sd}')
    print(f'Minute Volatility Absolute Change: {minute_vol_abs_change}')

    open_close_std = onemindf.close.groupby(onemindf.index.date).apply(lambda x: (x.iloc[-1] / x.iloc[0] - 1)).std()
    open_close_abs_change = onemindf.close.groupby(onemindf.index.date).apply(lambda x: (x.iloc[-1] / x.iloc[0] - 1)).abs().mean()

    print(f'Open Close SD: {open_close_std}')
    print(f'Open Close Absolute Change: {open_close_abs_change}')

    ratio_of_volatility = open_close_std / minute_vol_sd
    ratio_of_abs_change = open_close_abs_change / minute_vol_abs_change

    print(f'Ratio of Volatility: {ratio_of_volatility}')
    print(f'Ratio of Absolute Change: {ratio_of_abs_change}\n')

In [ ]:
# Determining the distribution of one min volatility
df = bnf_onemin
filtered_index = filter(lambda i: i.time() not in [time(9, 15), time(9, 16), time(15, 30)], df.index)
filtered_index = list(filtered_index)
filtered_df = df.close.pct_change()[filtered_index]
px.histogram(x=filtered_df)

# Modelling IV surface

In [ ]:
# Modelling IV surface
vol_surface = pd.read_csv('data/vol_surface.csv', index_col=0)
#vol_surface = vol_surface.drop(vol_surface[vol_surface.isna().all(axis=1)].index)
vol_surface['tte'] = vol_surface.time_to_expiry.apply(lambda num: round(num, 4))
vol_surface

In [ ]:
# Modelling IV surface
vol_surface_dict = {}
for tte in vol_surface.tte.unique():
    X = vol_surface.loc[vol_surface.tte == tte][['distance', 'distance_squared']]
    y = vol_surface.loc[vol_surface.tte == tte]['iv_multiple']
    model = LinearRegression()
    model.fit(X, y)
    dis_sq_coeff, dis_coeff, intercept = model.coef_[1], model.coef_[0], model.intercept_
    score = model.score(X, y)
    if score > 0.9:
        vol_surface_dict[tte] = {'dis_sq_coeff': dis_sq_coeff, 'dis_coeff': dis_coeff, 'intercept': intercept, 'score': score}
    # print(f'{tte} days to expiry: Coefficients: {model.coef_}, Intercept: {model.intercept_}, R2: {model.score(X, y)}')
vol_surface_weights = pd.DataFrame(vol_surface_dict).T.reset_index().rename(columns={'index': 'time_to_expiry'})
vol_surface_weights.sort_values('time_to_expiry', inplace=True)

In [ ]:
vol_surface_weights

In [ ]:
fig = px.scatter(vol_surface_weights, x='time_to_expiry', y='dis_sq_coeff')
fig.show()

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x) + c
lower_bounds = [-np.inf, -np.inf, -np.inf]
upper_bounds = [np.inf, np.inf, np.inf]
popt, pcov = curve_fit(func, vol_surface_weights['time_to_expiry'], vol_surface_weights['dis_sq_coeff'], bounds=(lower_bounds, upper_bounds))

In [ ]:
dummy_range = np.arange(0, 1, 0.0001)
fig.add_trace(px.line(x=dummy_range, y=func(dummy_range, *popt)).data[0])

In [ ]:
popt

In [ ]:
# Modelling IV surface - Distance Squared coefficient vs Time to Expiry (inverse)
for param in np.arange(0.02, 1.5, 0.01):
    vol_surface_weights['tte_inverse'] = 1 / (vol_surface_weights.time_to_expiry**param)
    X = vol_surface_weights['tte_inverse'].values.reshape(-1, 1)
    y = vol_surface_weights['dis_coeff']
    model = LinearRegression()
    model.fit(X, y)
    print(f'{param} param: Coefficients: {model.coef_}, Intercept: {model.intercept_}, R2: {model.score(X, y)}')

In [ ]:
px.scatter(vol_surface_weights, x='time_to_expiry', y='dis_coeff')

In [ ]:
def coefficients_for_surface(tte):

    # distance squared coefficient
    dfs2 = 3270.27*np.exp(-384.38*tte) + 100
    dfs2 = min(dfs2, 20000)

    # distance coefficient
    if tte < 0.26/365:
        dfs = 1
    else:
        dfs = 1 / ((tte ** 0.45) * 5)
        dfs = min(dfs, 5)
        dfs = -6 + dfs

    # intercept
    if tte<3/(24*365):
        intercept=1.07
    elif tte<0.27/365:
        intercept=1
    else:
        intercept=0.98
    return dfs2, dfs, intercept

In [ ]:
coefficients_for_surface(2/365)